In [1]:
from datetime import date, timedelta
import os
import pickle
import sys
import pandas as pd
sys.path.append("..")

In [2]:
# %pip install -U pandas

In [3]:
name = "rollback_analy"
# name = "inspiration_analy"
analy = pickle.load(
    open(os.path.join("..", "tests", "cache", f"{name}.p"), "rb" )
)
# df.head(2)

In [4]:
df = analy.get_basic_confidence_intervals()

In [5]:
from abautomator.visualizer import visualizer

from bokeh.io import output_notebook
from bokeh.plotting import show

output_notebook()

Loading BokehJS ...

In [6]:
df = analy.base_df.copy()

ctrl_df = analy._get_ctrl_df(df)
tx_df = analy._get_tx_df(df)

df = tx_df.merge(ctrl_df, how="left")
df["mean"] = df["tx_mean"] - df["ctrl_mean"]

result = analy._add_std_for_pop_proportion(df)

In [7]:
result

,exp_cond,metric,tx_mean,tx_std,tx_count,ctrl_mean,ctrl_std,ctrl_count,mean,std_ind,ctrl_succ,tx_succ,pooled_succ,pooled_count,pooled_prop,std_pooled
0,Tx,n_granted_contacts,0.192331,0.394141,20002.0,0.187399,0.390241,20331.0,0.004932,0.003906,3810.0,3847.0,7657.0,40333.0,0.189845,0.001953
1,Tx,pct_granted_contacts,0.192331,0.394141,20002.0,0.187399,0.390241,20331.0,0.004932,0.003906,3810.0,3847.0,7657.0,40333.0,0.189845,0.001953
2,Tx,n_entered_phone,0.484752,0.499780,20002.0,0.474005,0.499435,20331.0,0.010746,0.004975,9637.0,9696.0,19333.0,40333.0,0.479335,0.002488
3,Tx,pct_entered_phone,0.484752,0.499780,20002.0,0.473956,0.499334,20331.0,0.010795,0.004975,9636.0,9696.0,19332.0,40333.0,0.479310,0.002488


In [6]:
df = analy.get_basic_confidence_intervals()
df.head(10)

,exp_cond,metric,mean,std,count,upper_68_ci,lower_68_ci,upper_95_ci,lower_95_ci
0,Tx,n_granted_contacts,0.192331,0.394141,20002.0,0.586472,-0.201811,0.980613,-0.595952
1,Tx,pct_granted_contacts,0.192331,0.394141,20002.0,0.586472,-0.201811,0.980613,-0.595952
2,Control,n_granted_contacts,0.187399,0.390241,20331.0,0.577639,-0.202842,0.967880,-0.593083
3,Control,pct_granted_contacts,0.187399,0.390241,20331.0,0.577639,-0.202842,0.967880,-0.593083
4,Tx,n_entered_phone,0.484752,0.499780,20002.0,0.984531,-0.015028,1.484311,-0.514808
5,Tx,pct_entered_phone,0.484752,0.499780,20002.0,0.984531,-0.015028,1.484311,-0.514808
6,Control,n_entered_phone,0.474005,0.499435,20331.0,0.973440,-0.025429,1.472874,-0.524864
7,Control,pct_entered_phone,0.473956,0.499334,20331.0,0.973290,-0.025378,1.472623,-0.524711


In [7]:
p = visualizer.get_figure(analy.get_basic_confidence_intervals())
show(p)

cleaning


In [20]:
import scipy.stats
import numpy as np

In [21]:
df = analy.get_abs_diff_confidence_intervals()
df["metric"] = df["metric"].str.replace("_", " ")
df["metric"] = df["metric"].str.title()
df["metric"] = df["metric"].str.replace("Pct", "%")
df["factor_label"] = list(zip(df["metric"], df["exp_cond"]))

In [22]:
df.head()

,exp_cond,metric,tx_mean,tx_std,tx_count,factor_label,ctrl_mean,ctrl_std,ctrl_count,mean,std,upper_68_ci,lower_68_ci,upper_95_ci,lower_95_ci
0,Tx,N Granted Contacts,0.192331,0.394141,20002.0,"(N Granted Contacts, Tx)",0.187399,0.390241,20331.0,0.004932,0.003906,0.008838,0.001026,0.012744,-0.002880
1,Tx,% Granted Contacts,0.192331,0.394141,20002.0,"(% Granted Contacts, Tx)",0.187399,0.390241,20331.0,0.004932,0.003906,0.008838,0.001026,0.012744,-0.002880
2,Tx,N Entered Phone,0.484752,0.499780,20002.0,"(N Entered Phone, Tx)",0.474005,0.499435,20331.0,0.010746,0.004976,0.015722,0.005771,0.020697,0.000795
3,Tx,% Entered Phone,0.484752,0.499780,20002.0,"(% Entered Phone, Tx)",0.473956,0.499334,20331.0,0.010795,0.004975,0.015771,0.005820,0.020746,0.000845


In [25]:
df["t_calc"] = df["mean"]/df["std"]
df["t_crit"] = scipy.stats.t.ppf(q=1-.05/2, df=df["tx_count"] + df["ctrl_count"] - 2)
df["p_value"] = scipy.stats.t.sf(np.abs(df["t_calc"]), df["tx_count"] + df["ctrl_count"] - 2)*2  # two-sided pvalue = Prob(abs(t)>tt)
df.head()

,exp_cond,metric,tx_mean,tx_std,tx_count,factor_label,ctrl_mean,ctrl_std,ctrl_count,mean,std,upper_68_ci,lower_68_ci,upper_95_ci,lower_95_ci,t_calc,t_crit,p_value
0,Tx,N Granted Contacts,0.192331,0.394141,20002.0,"(N Granted Contacts, Tx)",0.187399,0.390241,20331.0,0.004932,0.003906,0.008838,0.001026,0.012744,-0.002880,1.262720,1.960023,0.206697
1,Tx,% Granted Contacts,0.192331,0.394141,20002.0,"(% Granted Contacts, Tx)",0.187399,0.390241,20331.0,0.004932,0.003906,0.008838,0.001026,0.012744,-0.002880,1.262720,1.960023,0.206697
2,Tx,N Entered Phone,0.484752,0.499780,20002.0,"(N Entered Phone, Tx)",0.474005,0.499435,20331.0,0.010746,0.004976,0.015722,0.005771,0.020697,0.000795,2.159808,1.960023,0.030793
3,Tx,% Entered Phone,0.484752,0.499780,20002.0,"(% Entered Phone, Tx)",0.473956,0.499334,20331.0,0.010795,0.004975,0.015771,0.005820,0.020746,0.000845,2.169912,1.960023,0.030019


In [15]:
p = visualizer.get_figure(analy.get_abs_diff_confidence_intervals())
show(p)

In [20]:
p = visualizer.get_figure(analy.get_rel_diff_confidence_intervals())
show(p)